<a href="https://colab.research.google.com/github/ohjiae/Hotel_canceled_booking_prediction/blob/main/hotel_canceled_booking_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **호텔 예약 취소 & 최적의 오버부킹 비율 예측 모델**
> About Dataset : [Kaggle 'Hotel booking Demand'](https://www.kaggle.com/jessemostipak/hotel-booking-demand)

## **데이터 개요**
- 리스본과 ㅇㅇ에 있는 호텔의 실제 데이터를 정리해 만든 데이터셋입니다.
- 실제 데이터이므로 일부 데이터는 익명화되어 있습니다.
- 데이터를 활용해 예약한 고객이 예약을 취소할지 예측해, 결론적으로 얼만큼 overbooking을 받는 것이 좋을지 알려주는 모델을 만들어 보겠습니다.

## 1) 데이터 및 패키지 불러오기

In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Data_Project/hotel_customer_segmentation/data
#!unzip -qq '/content/drive/MyDrive/Data_Project/hotel_customer_segmentation/data/archive.zip'

/content/drive/MyDrive/Data_Project/hotel_customer_segmentation/data


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

data = pd.read_csv('./hotel_bookings.csv')

## 2) 데이터 확인

◼ 데이터 

 [Hotel Booking Demand](https://www.kaggle.com/jessemostipak/hotel-booking-demand)

<br>

◼ 컬럼설명 

* hotel : 호텔의 종류 (H1 : Resort Hotel,  H2 : City Hotel)

* is_canceled : 예약 취소 여부 (1 : 취소 , 0 : 취소 x)

* lead_time :예약 후 도착할 때까지 시간

* arrival_date_year : 도착일(년)

* arrival_date_month : 도착일(월)

* arrival_date_week_number : 도착일(주차)

* arrival_date_day_of_month : 도착일(일)

* stays_in_weekend_nights : 주말 숙박일 수

* stays_in_week_nights : 평일 숙박일 수

* adults : 성인 수

* children : 어린이 수
 
* babies : 아기 수

* meal : 예약된 식사 유형 (BB, HB, SC, FB, Undefined)

* country : 나라

* market_segment : 시장유형 (Online TA, Offline TA/TO, Groups, Direct, Corporate, Complementary, Aviation, 
Undefined)

* distribution_channel : 유입 경로 (TA/TO : 여행사 투어, Direct, Corporate, GDS, Undefined)

* is_repeated_guest : 재방문 고객 (1 : 재방문)

* previous_cancellations : 이전예약 취소 수

* previous_bookings_not_canceled : 이전예약 미취소 수 

* reserved_room_type : 예약한 객실 유형 (A, B, C, D, E, F, G, H, L, P)

* assigned_room_type : 할당된 객실 유형 (A, B, C, D, E, F, G, H, I, K, L, P)

* booking_changes : 예약 변경 수

* deposit_type : 예약금 유형(No Deposit, Non Refund, Refundable)

* agent : 에이전트ID 

* company : 회사ID

* days_in_waiting_list : 예약 대기일 수

* customer_type : 고객 유형 (Transient, Transient-Party, Contract, Group)

* adr : 평균일일요금(=숙박요금/숙박일수)

* required_car_parking_spaces : 고객별 필요 주차공간

* total_of_special_requests : 고객의 특별 요구사항 수

* reservation_status : 마지막 상태 (Check-Out, Canceled, No-Show)

* reservation_status_date : 마지막 상태인 날짜

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [ ]:
display(data)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,0.0,0,BB,BEL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,394.0,NaN,0,Transient,96.14,0,0,Check-Out,2017-09-06
119386,City Hotel,0,102,2017,August,35,31,2,5,3,0.0,0,BB,FRA,Online TA,TA/TO,0,0,0,E,E,0,No Deposit,9.0,NaN,0,Transient,225.43,0,2,Check-Out,2017-09-07
119387,City Hotel,0,34,2017,August,35,31,2,5,2,0.0,0,BB,DEU,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.0,NaN,0,Transient,157.71,0,4,Check-Out,2017-09-07
119388,City Hotel,0,109,2017,August,35,31,2,5,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,89.0,NaN,0,Transient,104.40,0,0,Check-Out,2017-09-07


In [ ]:
# 컬럼내, 타입별 수
for col in data.select_dtypes(include=object):
  print(col)
  print(data[col].value_counts())

hotel
City Hotel      79330
Resort Hotel    40060
Name: hotel, dtype: int64
arrival_date_month
August       13877
July         12661
May          11791
October      11160
April        11089
June         10939
September    10508
March         9794
February      8068
November      6794
December      6780
January       5929
Name: arrival_date_month, dtype: int64
meal
BB           92310
HB           14463
SC           10650
Undefined     1169
FB             798
Name: meal, dtype: int64
country
PRT    48590
GBR    12129
FRA    10415
ESP     8568
DEU     7287
       ...  
BWA        1
CYM        1
PYF        1
MLI        1
AIA        1
Name: country, Length: 177, dtype: int64
market_segment
Online TA        56477
Offline TA/TO    24219
Groups           19811
Direct           12606
Corporate         5295
Complementary      743
Aviation           237
Undefined            2
Name: market_segment, dtype: int64
distribution_channel
TA/TO        97870
Direct       14645
Corporate     6677
GDS      

In [ ]:
# No show 예약 취소에 포함되는지 아닌지 확인
data[['reservation_status','is_canceled']].value_counts()
#k_data[['숙박 현황','예약 취소 여부']].value_counts()

reservation_status  is_canceled
Check-Out           0              75166
Canceled            1              43017
No-Show             1               1207
dtype: int64

In [ ]:
## 상관계수 분석
data.corr()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
is_canceled,1.000000,0.293123,0.016660,0.008148,-0.006130,-0.001791,0.024765,0.060017,0.005048,-0.032491,-0.084793,0.110133,-0.057358,-0.144381,-0.083114,-0.020642,0.054186,0.047557,-0.195498,-0.234658
lead_time,0.293123,1.000000,0.040142,0.126871,0.002268,0.085671,0.165799,0.119519,-0.037622,-0.020915,-0.124410,0.086042,-0.073548,0.000149,-0.069741,0.151464,0.170084,-0.063077,-0.116451,-0.095712
arrival_date_year,0.016660,0.040142,1.000000,-0.540561,-0.000221,0.021497,0.030883,0.029635,0.054624,-0.013192,0.010341,-0.119822,0.029218,0.030872,0.063457,0.259095,-0.056497,0.197580,-0.013684,0.108531
arrival_date_week_number,0.008148,0.126871,-0.540561,1.000000,0.066809,0.018208,0.015558,0.025909,0.005518,0.010395,-0.030131,0.035501,-0.020904,0.005508,-0.031201,-0.076760,0.022933,0.075791,0.001920,0.026149
arrival_date_day_of_month,-0.006130,0.002268,-0.000221,0.066809,1.000000,-0.016354,-0.028174,-0.001566,0.014544,-0.000230,-0.006145,-0.027011,-0.000300,0.010613,0.001487,0.044858,0.022728,0.030245,0.008683,0.003062
stays_in_weekend_nights,-0.001791,0.085671,0.021497,0.018208,-0.016354,1.000000,0.498969,0.091871,0.045793,0.018483,-0.087239,-0.012775,-0.042715,0.063281,0.140739,0.066749,-0.054151,0.049342,-0.018554,0.072671
stays_in_week_nights,0.024765,0.165799,0.030883,0.015558,-0.028174,0.498969,1.000000,0.092976,0.044203,0.020191,-0.097245,-0.013992,-0.048743,0.096209,0.182382,0.182211,-0.002020,0.065237,-0.024859,0.068192
adults,0.060017,0.119519,0.029635,0.025909,-0.001566,0.091871,0.092976,1.000000,0.030447,0.018146,-0.146426,-0.006738,-0.107983,-0.051673,-0.035594,0.207793,-0.008283,0.230641,0.014785,0.122884
children,0.005048,-0.037622,0.054624,0.005518,0.014544,0.045793,0.044203,0.030447,1.000000,0.024030,-0.032859,-0.024730,-0.021072,0.048949,0.041066,0.030931,-0.033273,0.324854,0.056253,0.081745
babies,-0.032491,-0.020915,-0.013192,0.010395,-0.000230,0.018483,0.020191,0.018146,0.024030,1.000000,-0.008943,-0.007501,-0.006550,0.083440,0.036184,0.019206,-0.010621,0.029186,0.037383,0.097889


In [ ]:
# 결측치 비중 확인 (data = 각 data 이름으로 변경해주세요)
for i in range(len(data.columns)):
    if data.iloc[:,i].isnull().sum()>0 :
        temp_col = data.iloc[:,i]
        percentage = temp_col.isnull().sum() / len(temp_col)
        print('{:}의 결측치는 전체의 {:.3%}입니다.'.format(temp_col.name,percentage))

children의 결측치는 전체의 0.003%%입니다.
country의 결측치는 전체의 0.409%%입니다.
agent의 결측치는 전체의 13.686%%입니다.
company의 결측치는 전체의 94.307%%입니다.


In [ ]:
# 데이터 프레임으로 변환
# 월 별 이름 숫자로 변환
#data.loc[data['arrival_date_month']=='July'].tail(10)
#month_dict = {}
#for i in data['arrival_date_month'].unique():

# 2. 데이터 전처리
## 1) 결측치 처리

In [ ]:
# 전체 결측치 확인
data.info()
# 결측치가 많은 열 삭제 > 전처리 이후 데이터 이름 'hotel'
hotel = data.drop(['agent','company'], axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 30 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [ ]:
# 결측치 확인
hotel.iloc[:,19:24].isnull().sum()

reserved_room_type      0
assigned_room_type      0
booking_changes         0
deposit_type            0
days_in_waiting_list    0
dtype: int64

In [ ]:
# 이상치 확인 (unique 값 정리해서 dict로 만들기)
uniq_dict = {}
for i in hotel.iloc[:,19:24].columns:
    temp = list(hotel[i].unique())
    uniq_dict[i]= sorted(temp)

for key, value in uniq_dict.items():
    print(key,':',value)

reserved_room_type : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'L', 'P']
assigned_room_type : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'P']
booking_changes : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21]
deposit_type : ['No Deposit', 'Non Refund', 'Refundable']
days_in_waiting_list : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 79, 80, 81, 83, 84, 85, 87, 89, 91, 92, 93, 96, 97, 98, 99, 100, 101, 105, 107, 108, 109, 111, 113, 116, 117, 120, 121, 122, 125, 142, 147, 150, 154, 160, 162, 165, 167, 174, 175, 176, 178, 183, 185, 187, 193, 207, 215, 223, 224, 236, 259, 330, 379, 391]
